In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random as rng
import math
import datetime as time

In [21]:
class Calculator :
    name = ""
    components = []
    
    loadTotal = 0
    prodTotal = 0
    battTotal = 0
    
    chargeRate = 0
    chargeTime_sec = 0
    chargeTime_time = time
    
    def __init__(self, name) :
        self.clear()
        self.name = name
        
    def addComponent(self, name, loadEc, prodEc, batEc, loadPer="sec", prodPer="sec", multi=1, rwEc=0, rwPer="sec") :
        temp = Component(name, loadEc, prodEc, batEc, loadPer, prodPer, multi, rwEc, rwPer)
        temp.calcSeconds()
        self.components.append(temp)
        
    
    def calculate(self) :
        self.calculateComponents()
        self.calcChargeRate()
        self.calcChargeTime()
        self.convertTime()
        
    def calculateComponents(self) :
        for comp in self.components :
            self.loadTotal += comp.load.ecPerSec * comp.multi
            self.prodTotal += comp.prod.ecPerSec * comp.multi
            self.battTotal += comp.batt.ec * comp.multi
            
    def calcChargeRate(self) :
        self.chargeRate = self.prodTotal - self.loadTotal
        
    def calcChargeTime(self) :
        if self.chargeRate != 0 :
            self.chargeTime_sec = self.battTotal / self.chargeRate
        else :
            self.chargeTime_sec = 0
            
    def convertTime(self) :
        self.chargeTime_time = time.timedelta(seconds=self.chargeTime_sec)
        
    def calculateAntennaPulse(self, component, seconds) :
        return component.load.ecPerSec * seconds
            
    def clear(self) :
        self.name = ""
        self.components.clear()
        
        self.loadTotal = 0
        self.prodTotal = 0
        self.battTotal = 0
        
        self.chargeRate = 0
        self.chargeTime_sec = 0
        self.chargeTime_time = time
        
    def ToString(self) :
        components = ""
        for c in self.components :
            components += f"{c.ToString()}\n"
        
        output = f"""
{self.name}
Load : {round(self.loadTotal, 6)}
Prod : {round(self.prodTotal, 6)}
Batt : {round(self.battTotal, 6)}
        
Charge Rate : {round(self.chargeRate, 6)}
Charge Secs : {round(self.chargeTime_sec, 6)}
Charge Time : {self.chargeTime_time}
        
{components}"""
        
        return output

class Component :
    name = ""
    load = None
    prod = None
    batt = None
    multi = 1
    
    def __init__(self, name, loadEc, prodEc, batEc, loadPer="sec", prodPer="sec", multi=1, rwEc=0, rwPer="sec") :
        self.name = name
        self.load = Load(loadEc, rwEc, loadPer, rwPer)
        self.prod = Production(prodEc, per=prodPer)
        self.batt = Battery(batEc)
        self.multi = multi
        
    def calcSeconds(self) :
        if self.load != None :
            self.load.calcSeconds()
        if self.prod != None :
            self.prod.calcSeconds()
            
    def ToString(self) :
        return f"{self.name} : {self.load.ToString()} : {self.prod.ToString()} : {self.batt.ToString()} : Count {self.multi}"
    
    
class Base :
    ecPerSec = 0
    
    def calcSeconds(self) :
        if self.per == "min" :
            self.ecPerSec = self.ec / 60
        elif self.per == "hr" :
            self.ecPerSec = self.ec / math.pow(60, 2)
        else :
            self.ecPerSec = self.ec
        
        if type(self) == Load :
            if self.rwEc != 0 :
                if self.rwPer == "min" :
                    self.ecPerSec += self.rwEc / 60
                elif self.rwPer == "hr" :
                    self.ecPerSec += self.rwEc / math.pow(60, 2)
                else :
                    self.ecPerSec += self.rwEc
    
class Load(Base) :
    ec = 0
    per = "sec"
    rwEc = 0
    rwPer = "sec"
    
    def __init__(self, ec, rwEc=0, per="sec", rwPer="sec") :
        self.calcSeconds()
        self.ec = ec
        self.per = per
        self.rwEc = rwEc
        self.rwPer = rwPer
    
    def ToString(self) :
        if self.rwEc != 0 :
            rwString = f" , RW Load {self.rwEc} ec/{self.rwPer}"
        else :
            rwString = ""
        return f"Load {round(self.ecPerSec, 6)} ec/sec , {self.ec}ec/{self.per}{rwString}"
    
class Production(Base) :
    ec = 0
    per = "sec"
    
    def __init__(self, ec, per) :
        self.calcSeconds()
        self.ec = ec
        self.per = per
        
    
    def ToString(self) :
        return f"Prod {round(self.ecPerSec, 6)} ec/sec , EC {self.ec} ec/{self.per}"
        
class Battery :
    ec = 0
    def __init__(self, ec) :
        self.ec = ec
        
    
    def ToString(self) :
        return f"Batt {self.ec} ec"

In [22]:
# addComponent(name, loadEc, prodEc, batEc, loadPer="sec", prodPer="sec", multi=1, rwEc=0, rwPer="sec") :

In [23]:
calc1 = Calculator("Test 1")

calc1.addComponent("Processor", 0.04, 0, 5)
calc1.addComponent("Command", 1.2, 0, 0, loadPer="min", rwEc=1.8, rwPer="min")
calc1.addComponent("scan Biome", 1, 0, 0)
calc1.addComponent("scan Alt", 6, 0,0,loadPer="min")

calc1.addComponent("small solar", 0, 0.3, 0, prodPer="sec", multi=4)
calc1.addComponent("small Batt R", 0,0,200, multi=2)

calc1.calculate()

In [24]:
print(calc1.ToString())


Test 1
Load : 1.19
Prod : 1.2
Batt : 405
        
Charge Rate : 0.01
Charge Secs : 40500.0
Charge Time : 11:15:00
        
Processor : Load 0.04 ec/sec , 0.04ec/sec : Prod 0 ec/sec , EC 0 ec/sec : Batt 5 ec : Count 1
Command : Load 0.05 ec/sec , 1.2ec/min , RW Load 1.8 ec/min : Prod 0 ec/sec , EC 0 ec/sec : Batt 0 ec : Count 1
scan Biome : Load 1 ec/sec , 1ec/sec : Prod 0 ec/sec , EC 0 ec/sec : Batt 0 ec : Count 1
scan Alt : Load 0.1 ec/sec , 6ec/min : Prod 0 ec/sec , EC 0 ec/sec : Batt 0 ec : Count 1
small solar : Load 0 ec/sec , 0ec/sec : Prod 0.3 ec/sec , EC 0.3 ec/sec : Batt 0 ec : Count 4
small Batt R : Load 0 ec/sec , 0ec/sec : Prod 0 ec/sec , EC 0 ec/sec : Batt 200 ec : Count 2



In [25]:
testLoad = Load(2, rwEc=1, per="min", rwPer="sec")
testProd = Production(1, "min")

print(type(testLoad))
print(type(testProd))

if type(testLoad) == Load :
    print("yes")
else :
    print("no")

<class '__main__.Load'>
<class '__main__.Production'>
yes


In [28]:
calc2 = Calculator("ElectTest1")

calc2.addComponent("HECS2 Command", 3, 0, 0, loadPer="min", rwEc=0.5, rwPer="sec")
calc2.addComponent("SCAN Biome", 1,0,0)
calc2.addComponent("Z-1K Battery", 0,0,1000)
calc2.addComponent("Medium RW", 0,0,0, rwEc=0.5)
calc2.addComponent("OX-4L 1x6", 0,1.6,0, multi=2)

calc2.calculate()

In [29]:
print(calc2.ToString())


ElectTest1
Load : 2.05
Prod : 3.2
Batt : 1000
        
Charge Rate : 1.15
Charge Secs : 869.565217
Charge Time : 0:14:29.565217
        
HECS2 Command : Load 0.55 ec/sec , 3ec/min , RW Load 0.5 ec/sec : Prod 0 ec/sec , EC 0 ec/sec : Batt 0 ec : Count 1
SCAN Biome : Load 1 ec/sec , 1ec/sec : Prod 0 ec/sec , EC 0 ec/sec : Batt 0 ec : Count 1
Z-1K Battery : Load 0 ec/sec , 0ec/sec : Prod 0 ec/sec , EC 0 ec/sec : Batt 1000 ec : Count 1
Medium RW : Load 0.5 ec/sec , 0ec/sec , RW Load 0.5 ec/sec : Prod 0 ec/sec , EC 0 ec/sec : Batt 0 ec : Count 1
OX-4L 1x6 : Load 0 ec/sec , 0ec/sec : Prod 1.6 ec/sec , EC 1.6 ec/sec : Batt 0 ec : Count 2

